In [1]:
import pandas as pd
import numpy as np
import mysql.connector
import os
from mysql.connector import Error
from datetime import datetime, date, timedelta
import re
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
#Definimos la ruta en donde queremos guardar los archivos
ruta_guardar = '/home/estadistico/Documents/Erick/Reportes diarios'
ruta_pagos = '/home/estadistico/Documents/Erick'
mes = 20201201
mes2 = 'Diciembre2020'
mes3 = pd.to_datetime('01/12/2020',dayfirst=True)
prim_sem_mes = 49

In [3]:
#Agregamos las variables a ocupar
servidor = '192.168.15.12'
puerto = 3306
usuario = 'estadisticas'
contrasena = 'estadisticas8474'
base = 'procesos_externos'

In [4]:
#Asignamos valores a los parametros \n"
today = date.today().strftime('%Y%m%d')
now = datetime.now().strftime('%d-%m-%Y %H:%M')
hoy = np.datetime64('today')
ayer = np.datetime64(date.today() - timedelta(1))
today2 = date.today().strftime('%d-%m-%Y')

In [5]:
#Lectura pagos Banco Azteca
sheet_baz_renta = os.listdir(''+ruta_pagos+'/'+str(mes2)+'/Banco Azteca/Pagos/Renta')
pagos_baz_renta = pd.DataFrame()
for i in sheet_baz_renta:
    pagos_baz_renta_new = pd.read_csv(''+ruta_pagos+'/'+str(mes2)+'/Banco Azteca/Pagos/Renta/'+i+'')
    pagos_baz_renta = pd.concat([pagos_baz_renta,pagos_baz_renta_new],ignore_index=True)
pagos_baz_renta = pagos_baz_renta.drop_duplicates()
pagos_baz_renta = pagos_baz_renta.loc[pagos_baz_renta['FNMONTOPAGADO']>0]
pagos_baz_renta['FDFECHAABONO'] = pd.to_datetime(pagos_baz_renta['FDFECHAABONO'],dayfirst=True)
pagos_baz_renta = pagos_baz_renta.loc[pagos_baz_renta['FDFECHAABONO']>=mes3]
pagos_baz_renta2 = pd.pivot_table(pagos_baz_renta,index=['CAMPANAID','FDFECHAABONO','FIIDPERIODO','FNSCOMPROMISO','FCEMPNUMCORTE','FNMONTOPROMETIDO'],values=['FNMONTOPAGADO'],aggfunc=np.sum)
pagos_baz_renta2 = pd.DataFrame(pagos_baz_renta2.to_records())

In [6]:
#Lectura pagos Banco Azteca
sheet_baz_recovery = os.listdir(''+ruta_pagos+'/'+str(mes2)+'/Banco Azteca/Pagos/Recovery/')
pagos_baz_recovery = pd.DataFrame()
for k in sheet_baz_recovery:
    pagos_baz_recovery_new = pd.read_csv(''+ruta_pagos+'/'+str(mes2)+'/Banco Azteca/Pagos/Recovery/'+k+'',sep='|')
    pagos_baz_recovery = pd.concat([pagos_baz_recovery,pagos_baz_recovery_new],ignore_index=True)
pagos_baz_recovery['Cartera'] = 'Recovery'
pagos_baz_recovery['DIA'] = pd.to_datetime(pagos_baz_recovery['DIA'],dayfirst=True)
pagos_baz_recovery = pagos_baz_recovery.loc[pagos_baz_recovery['DIA']>=mes3]
pagos_baz_recovery = pagos_baz_recovery.loc[pagos_baz_recovery['DIA']>=mes3]
pagos_baz_recovery['credito'] = pagos_baz_recovery['PAIS'].map(str)+'-'+pagos_baz_recovery['CANAL'].map(str)+'-'+pagos_baz_recovery['SUCURSAL'].map(str)+'-'+pagos_baz_recovery['FOLIO'].map(str)
pagos_baz_recovery = pagos_baz_recovery.drop_duplicates()

In [7]:
#Lectura pagos Banco Azteca
sheet_baz_26 = os.listdir(''+ruta_pagos+'/'+str(mes2)+'/Banco Azteca/Pagos/26-39/')
pagos_baz_26 = pd.DataFrame()
for j in sheet_baz_26:
    pagos_baz_26_new = pd.read_csv(''+ruta_pagos+'/'+str(mes2)+'/Banco Azteca/Pagos/26-39/'+j+'',sep='|')
    pagos_baz_26 = pd.concat([pagos_baz_26,pagos_baz_26_new],ignore_index=True)
pagos_baz_26['cartera'] = '26-39'
pagos_baz_26['DIA'] = pd.to_datetime(pagos_baz_26['DIA'],dayfirst=True)
pagos_baz_26 = pagos_baz_26.loc[pagos_baz_26['DIA']>=mes3]
pagos_baz_26['credito'] = pagos_baz_26['PAIS'].map(str)+'-'+pagos_baz_26['CANAL'].map(str)+'-'+pagos_baz_26['SUCURSAL'].map(str)+'-'+pagos_baz_26['FOLIO'].map(str)
pagos_baz_26 = pagos_baz_26.drop_duplicates()
pagos_baz_26.head(1)

,ANIO,SEMANA,DIA,PAIS,CANAL,SUCURSAL,FOLIO,RECUPERACION DE CAPITAL,RECUPERACION DE MORATORIOS,SALDO ACTUAL,MORATORIOS ACTUAL,FECHA GESTION,CARGO AUTOMATICO,cartera,credito
0,2020,50,2020-12-07,1,1,4167,21140,373,632,5028.0,0.0,02/12/2020,NO,26-39,1-1-4167-21140


In [8]:
#Hacemos la conexion con el servidor\n",
try: 
    conn = mysql.connector.connect(user=usuario,
                               password=contrasena,
                               host=servidor,
                               port=puerto,
                               database=base)
    conn.set_charset_collation('latin1')
except mysql.Error as e:
    print("Failed to execute stored procedure: {}".format(error))

cursor = conn.cursor()

In [9]:
#Funcion para hacer la consulta sql, ya sea de gestion o de asignacion en donde requerimos tres parametros, la tabla de donde vamos a sacar la consulta, en caso de asignacin requerimos los otros dos parametros fecha inicial y fecha final
def consulta_sql(table,from_date=None,to_date=None,campana=None):
    if campana is None:
        if from_date is None:
            sql_mes = cursor.callproc(table,)
            for result in cursor.stored_results():
                sql_mes = pd.DataFrame(result.fetchall())
        else:
            sql_mes = cursor.callproc(table, [from_date,to_date])
            for result in cursor.stored_results():
                sql_mes = pd.DataFrame(result.fetchall())
    else:
        sql_mes = cursor.callproc(table, [from_date,to_date,campana])
        for result in cursor.stored_results():
                sql_mes = pd.DataFrame(result.fetchall())
    sql_mes = sql_mes.drop_duplicates()
    return sql_mes

In [10]:
#Funcion para transformar varias columnas a numero
def transform_numeric(df,cols):
    return df[cols].apply(pd.to_numeric, errors='coerce')

#Funcion que convierte a dataframe y rellena espacios con 0
def to_dataframe(df):
    df = pd.DataFrame(df.to_records())
    df.fillna(0,inplace=True)
    return df

#Funcion que quita el % del string
def percent_tonumeric(df,cols):
    for c in cols:
        df[c] = [x[:-2] for x in df[c]]
    return transform_numeric(df,cols)

# Llamadas Detallado Banco Azteca

In [11]:
#Hacemos la consulta referente a las gestiones de Credifiel
#llamadas_cred = consulta_sql('procesos_externos.all_rpt_llamadas_detallado',mes,today,'(^BAZ)')
#llamadas_cred.columns = ['server_ip','uniqueid','call_type','call_date','campaign_id','campaign_name','list_id',#'list_name','lead_id','vendor_lead_id','vendor_lead_name','phone_number','status','status_name','username','full_name','term_reason_call','queue_seconds','length_in_sec','call_date_short','call_time_short','call_time_hour','call_range_hour','dialstatus','hangup_cause','sip_hangup_cause','carrier','areacode','state','region']
#llamadas_cred.head(1)

In [12]:
#llamadas_cred['turno'] = np.where(llamadas_cred['call_time_hour']<=14,'M','V')

# Performance Banco Azteca

In [13]:
#Hacemos la consulta referente al performance del Gestor
performance_all = consulta_sql('procesos_externos.all_rpt_agent_performance',20200101,20201201)
performance_all.columns = ['fecha','usuario','nombre_usuario','user_group','llamadas','aht','ocupacion','adherencia','login_vici','logout_vici','staff_time','wait_time','talk_time','dispo_time','pause','login','manual','gestion','agua','alimentos','sanitario','retroalimentacion','tranferencia','validacion','whatsapp','incumplimiento','recordatorios','capacitacion','sistemas','lagged','tipificando','*']

In [14]:
performance_all['user_group2'] = [x[:3] for x in performance_all['user_group']]
performance_baz = performance_all.loc[performance_all['user_group2']=='BAZ']
performance_baz[['ocupacion','adherencia']] = percent_tonumeric(performance_baz,['ocupacion','adherencia'])

In [15]:
performance_baz['hora_login'] = [int(x[:2]) for x in performance_baz['login_vici']]
performance_baz['hora_logout'] = [int(x[:2]) for x in performance_baz['logout_vici']]
performance_baz['turno'] = np.where((performance_baz['hora_logout']<=14)&(performance_baz['hora_logout']-performance_baz['hora_login']<=8),'M',
                                    np.where((performance_baz['hora_logout']>17)&(performance_baz['hora_logout']-performance_baz['hora_login']<=8),'V','Mixto'))

In [16]:
performance_baz['horas_trabajadas'] = performance_baz['hora_logout']-performance_baz['hora_login']
performance_baz['llamadas_hora'] = performance_baz['llamadas']/performance_baz['horas_trabajadas']
performance_baz['llamadas_hora'] = performance_baz['llamadas_hora'].round(1)

In [33]:
control_performance_baz_turno = pd.pivot_table(performance_baz,index=['turno'],values=['llamadas','ocupacion','horas_trabajadas'],aggfunc=[np.sum,np.mean,'count',min,max])
control_performance_baz_turno = to_dataframe(control_performance_baz_turno)
#control_performance_baz_turno = control_performance_baz_turno.iloc[:,[0,1,2,3,4,5,6,7,8,9,12,13,14,15,16,17]]
#control_performance_baz_turno.columns = ['Usuario','Gestor','Turno','SumHorasTrabajadas','SumLlamadasDia','SumOcupacion','PromedioHorasTrab','PromedioLlamadasDia','PromedioOcupacion','DiasAsitencia','MinHoras','MinLlamadasDia','MinOcupacion','MaxHoras','MaxLlamadasDia','MaxOcupacion']
#control_performance_baz_turno['LlamadasHora'] = control_performance_baz_turno['SumLlamadasDia']/control_performance_baz_turno['SumHorasTrabajadas']
control_performance_baz_turno

,turno,"('sum', 'horas_trabajadas')","('sum', 'llamadas')","('sum', 'ocupacion')","('mean', 'horas_trabajadas')","('mean', 'llamadas')","('mean', 'ocupacion')","('count', 'horas_trabajadas')","('count', 'llamadas')","('count', 'ocupacion')","('min', 'horas_trabajadas')","('min', 'llamadas')","('min', 'ocupacion')","('max', 'horas_trabajadas')","('max', 'llamadas')","('max', 'ocupacion')"
0,M,15510,842920,100472.06,5.960799,323.950807,38.643100,2602,2602,2600,0,0,0.0,8,1390,85.16
1,Mixto,1564,55547,6563.34,9.309524,330.636905,39.777818,168,168,165,0,0,0.0,14,966,76.69
2,V,11463,566965,71321.73,6.691769,330.977817,41.635569,1713,1713,1713,0,0,0.0,8,1208,74.15


In [34]:
control_performance_baz_turno3 = control_performance_baz_turno[np.abs(control_performance_baz_turno - 
control_performance_baz_turno.mean()) <= (3 * control_performance_baz_turno.std())]
control_performance_baz_turno3 = control_performance_baz_turno3.replace({'0':np.nan, 0:np.nan})

In [35]:
control_performance_baz_turno3.loc['mean'] = control_performance_baz_turno3.mean().round(2)
control_performance_baz_turno3

,turno,"('sum', 'horas_trabajadas')","('sum', 'llamadas')","('sum', 'ocupacion')","('mean', 'horas_trabajadas')","('mean', 'llamadas')","('mean', 'ocupacion')","('count', 'horas_trabajadas')","('count', 'llamadas')","('count', 'ocupacion')","('min', 'horas_trabajadas')","('min', 'llamadas')","('min', 'ocupacion')","('max', 'horas_trabajadas')","('max', 'llamadas')","('max', 'ocupacion')"
0,NaN,15510.00,842920.00,100472.06,5.960799,323.950807,38.643100,2602.00,2602.00,2600.00,NaN,NaN,NaN,8.0,1390.0,85.16
1,NaN,1564.00,55547.00,6563.34,9.309524,330.636905,39.777818,168.00,168.00,165.00,NaN,NaN,NaN,14.0,966.0,76.69
2,NaN,11463.00,566965.00,71321.73,6.691769,330.977817,41.635569,1713.00,1713.00,1713.00,NaN,NaN,NaN,8.0,1208.0,74.15
mean,NaN,9512.33,488477.33,59452.38,7.320000,328.520000,40.020000,1494.33,1494.33,1492.67,NaN,NaN,NaN,10.0,1188.0,78.67


# Gestion Banco Azteca

In [20]:
#Inicio y fin de meses que nos interesa conocer
inicio_mes = [20200101,20200201,20200301,20200401,20200501,20200601,20200701,20200801,20200901,20201001,20201101]
fin_mes = [20200131,20200228,20200331,20200430,20200531,20200630,20200731,20200831,20200930,20201031,20201130]

In [21]:
#Lectura de cada uno de los meses
gestion_baz = pd.DataFrame()
for i,j in zip(inicio_mes,fin_mes):
    gestion_baz_mes = consulta_sql('baz_rpt_gestiones_detallado',i,j)
    gestion_baz = pd.concat([gestion_baz,gestion_baz_mes],ignore_index=True)
    print(i)

20200101
20200201
20200301
20200401
20200501
20200601
20200701
20200801
20200901
20201001
20201101


In [22]:
#Hacemos la consulta referente a las gestiones de Banco Azteca
gestion_baz.columns = ['folio_gestion','firma_id','unegocio_id','credito','nombre_credito','telefono','tipo_telefono','fecha_gestion','usuario','nombre_usuario','dictamen','accion_resultado','fecha_promesa','monto_promesa','comentarios']

In [23]:
#try: gestion_baz['anio_prom'] = [int(x[:4]) for x in gestion_baz['fecha_promesa']]
#except: gestion_baz['anio_prom'] = 0
#gestion_baz = gestion_baz.loc[gestion_baz['anio_prom']<2030]
gestion_baz = gestion_baz.drop_duplicates()
gestion_baz['monto_promesa'] = pd.to_numeric(gestion_baz['monto_promesa'])
gestion_baz['usuario'] = [x.rstrip() for x in gestion_baz['usuario']]
#gestion_baz['fecha_promesa'] = pd.to_datetime(gestion_baz['fecha_promesa'],dayfirst=False)

In [25]:
promesas_baz = gestion_baz.loc[gestion_baz['dictamen']=='PROMESA']
promesas_baz = promesas_baz.loc[(promesas_baz['monto_promesa']>10) & (promesas_baz['monto_promesa']<500000)]

In [37]:
pagos_baz = pd.concat([pagos_baz_26,pagos_baz_recovery],ignore_index=True)
pagos_baz = pagos_baz.loc[pagos_baz['CARGO AUTOMATICO']=='NO']
consolidado_pagos_baz = pd.merge(promesas_baz,pagos_baz,how='left',on=['credito'])
consolidado_pagos_baz['MONTH'] = consolidado_pagos_baz['fecha_gestion'].dt.month
consolidado_pagos_baz.head(1)

,folio_gestion,firma_id,unegocio_id,credito,nombre_credito,telefono,tipo_telefono,fecha_gestion,usuario,nombre_usuario,...,FOLIO,RECUPERACION DE CAPITAL,RECUPERACION DE MORATORIOS,SALDO ACTUAL,MORATORIOS ACTUAL,FECHA GESTION,CARGO AUTOMATICO,cartera,Cartera,MONTH
0,6604661,7,1,1-1-3625-80005,MONICA BERENICE ARRIAGA RAMIREZ,5561920651,celular,2020-01-31 20:27:36,GACASTAÃEDAGA,GALLO CASTAÃEDA GUSTAVO ADOLFO,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1


In [44]:
promesas_gestor_rec = pd.pivot_table(consolidado_pagos_baz,index=['usuario'],values=['monto_promesa'],columns=['MONTH'],aggfunc=[np.sum])
promesas_gestor_rec = to_dataframe(promesas_gestor_rec)
promesas_gestor_rec = promesas_gestor_rec.iloc[:,:-1]

#promesas_gestor_rec.columns = ['Usuario','NumeroPromesas','SumaRecCapital','SumaRecMoratorios','SumaPromesas']
#promesas_gestor_rec['Recuperado'] = promesas_gestor_rec['SumaRecCapital']+promesas_gestor_rec['SumaRecMoratorios']
promesas_gestor_rec

,usuario,"('sum', 'monto_promesa', 1)","('sum', 'monto_promesa', 2)","('sum', 'monto_promesa', 3)","('sum', 'monto_promesa', 4)","('sum', 'monto_promesa', 5)","('sum', 'monto_promesa', 6)","('sum', 'monto_promesa', 7)","('sum', 'monto_promesa', 8)","('sum', 'monto_promesa', 9)","('sum', 'monto_promesa', 10)",Mean
0,ACDELRIOCH,0.00,0.0,0.00,0.00,0.00,0.00,0.00,2156.00,0.00,0.00,215.60
1,ADAMECCAL,0.00,56306.0,370314.00,368627.00,0.00,0.00,0.00,0.00,0.00,0.00,79524.70
2,ALHENANDEZA,416428.00,350149.0,478852.00,361515.00,182027.00,325670.00,323915.00,464247.00,357875.00,159559.00,342023.70
3,ALLENDEIGCRJO,0.00,0.0,0.00,0.00,0.00,0.00,4100.00,58752.00,0.00,0.00,6285.20
4,ALVARADOCHCI,42208.00,5949.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,4815.70
...,...,...,...,...,...,...,...,...,...,...,...,...
161,VIGARCIAEJ,0.00,0.0,0.00,0.00,0.00,0.00,0.00,9111.00,150773.00,6734.00,16661.80
162,VIRELASTOJOAN,0.00,0.0,0.00,0.00,0.00,0.00,0.00,0.00,84563.00,269456.00,35401.90
163,WJZCARRANCO,0.00,0.0,0.00,0.00,35980.00,0.00,0.00,0.00,0.00,0.00,3598.00
164,ZEPEDAESJI,0.00,0.0,0.00,0.00,0.00,0.00,0.00,4810.00,122508.00,278588.00,40590.60


In [45]:
promesas_gestor_rec = promesas_gestor_rec[np.abs(promesas_gestor_rec - 
promesas_gestor_rec.mean()) <= (3 * promesas_gestor_rec.std())]
promesas_gestor_rec['Mean'] = promesas_gestor_rec.mean(numeric_only=True, axis=1).round(2)
promesas_gestor_rec.loc['mean'] = promesas_gestor_rec.mean().round(2)
promesas_gestor_rec = promesas_gestor_rec.replace({'0':np.nan, 0:np.nan})
promesas_gestor_rec

,usuario,"('sum', 'monto_promesa', 1)","('sum', 'monto_promesa', 2)","('sum', 'monto_promesa', 3)","('sum', 'monto_promesa', 4)","('sum', 'monto_promesa', 5)","('sum', 'monto_promesa', 6)","('sum', 'monto_promesa', 7)","('sum', 'monto_promesa', 8)","('sum', 'monto_promesa', 9)","('sum', 'monto_promesa', 10)",Mean
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2156.00,NaN,NaN,215.60
1,NaN,NaN,56306.00,370314.00,368627.00,NaN,NaN,NaN,NaN,NaN,NaN,79524.70
2,NaN,416428.00,350149.00,478852.00,361515.00,182027.00,325670.00,323915.00,NaN,NaN,159559.00,324764.38
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4100.00,58752.00,NaN,NaN,6285.20
4,NaN,42208.00,5949.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4815.70
...,...,...,...,...,...,...,...,...,...,...,...,...
161,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9111.00,150773.00,6734.00,16661.80
162,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,84563.00,269456.00,35401.90
163,NaN,NaN,NaN,NaN,NaN,35980.00,NaN,NaN,NaN,NaN,NaN,3598.00
164,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4810.00,122508.00,278588.00,40590.60


In [47]:
writer = pd.ExcelWriter(''+ruta_guardar+'/Gestores/Banco Azteca/Puntos Banco Azteca '+now+'.xlsx',engine='xlsxwriter')
promesas_gestor_rec.to_excel(writer,'BSC',index=False,header=True)
control_performance_baz_turno.to_excel(writer,'PerformanceDetallado',index=False,header=True)
control_performance_baz_turno3.to_excel(writer,'PerformanceTurno',index=False,header=True)

writer.save()